# Self-reflection pattern

Self-reflection pattern contains at least 2 promtps: the prompt that perform task itself and prompt to reflect the previous response. In self-reflection pattern, 2 prompt are performed by same LLM model in separated fashion.

In this example, we are using langchain API and an ollama model hosted locally.

## Setup
host an ollama instance by using either running a docker or a run a service following instruction from ollama's guide.

In [1]:
import sys
sys.path.append("../../src/")

In [ ]:
from agent_design_pattern.agent import AgentMessage
from agent_design_pattern.orchestration import ReflectionAgent

from chain import CasualOllamaSingleTurnChain

In [4]:
system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
- The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code
"""
user_prompt_task = "{query}"
task_chain = CasualOllamaSingleTurnChain("qwen3:4b-thinking-2507-q4_K_M", system_prompt_task, user_prompt_task, base_url="192.168.55.1:11434")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain full function code, inline code comment and docstring, nothing else."""
user_prompt_reflection = "Input query: {query}\n\nFunction implementation: {context_response}"
# Use the same llm for task and self-reflection
reflection_chain = CasualOllamaSingleTurnChain(task_chain.llm, system_prompt_reflection, user_prompt_reflection, base_url="192.168.55.1:11434")  # use the same model for self-reflection

def state_callback(state: str):
    print(f"agent state: {state}")
reflection_agent = ReflectionAgent(task_chain, reflection_chain, state_change_callback=state_callback)

In [5]:
# Take a leetcode as an example. Source: https://leetcode.com/problems/palindrome-partitioning-ii/description/
query = """Write python function(s) to solve the following problem:
Given a string s, partition s such that every substring of the partition is a palindrome.
Return the minimum cuts needed for a palindrome partitioning of s.

Example 1:
Input: s = "aab"
Output: 1
Explanation: The palindrome partitioning ["aa","b"] could be produced using 1 cut.

Example 2:
Input: s = "a"
Output: 0

Example 3:
Input: s = "ab"
Output: 1

Constraints:
1 <= s.length <= 2000
s consists of lowercase English letters only."""

final_message = reflection_agent.execute(AgentMessage(query=query))
print(final_message)
print("final response")
print(final_message.response)

agent state: running
agent state: reflecting
agent state: idle
query='Write python function(s) to solve the following problem:\nGiven a string s, partition s such that every substring of the partition is a palindrome.\nReturn the minimum cuts needed for a palindrome partitioning of s.\n\nExample 1:\nInput: s = "aab"\nOutput: 1\nExplanation: The palindrome partitioning ["aa","b"] could be produced using 1 cut.\n\nExample 2:\nInput: s = "a"\nOutput: 0\n\nExample 3:\nInput: s = "ab"\nOutput: 1\n\nConstraints:\n1 <= s.length <= 2000\ns consists of lowercase English letters only.' origin='ReflectionAgent_1' response='\n\nThe provided function implementation correctly solves the problem of finding the minimum number of cuts required to partition a string into palindromic substrings. The code follows the problem specifications and adheres to Google style docstrings with clear examples. After thorough review, **no errors were found** in the implementation.\n\n```python\ndef minCut(s: str) -> i